In [ ]:
## working in Spanish, but talks a lot of non-sense...

In [2]:
#!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
#!pip install -q datasets bitsandbytes einops wandb


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
/usr/local/lib/python3.10/dist-packages/transformer_engine_extensions.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

In [4]:
model_id  = "aguila"

tokenizer = AutoTokenizer.from_pretrained(model_id)
generator = pipeline(
    "text-generation",
    #'conversational',
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/projecte-aina/aguila-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/projecte-aina/aguila-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.06s/it]
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'Camembert

In [ ]:
input_text = "El mercat del barri és fantàstic, hi pots trobar"

In [ ]:
generation = generator(
    input_text,
    do_sample=True,
    top_k=10,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
print(f"Result: {generation[0]['generated_text']}")

In [ ]:
input_text = "User: hola, cómo estás?\nMini:"

In [ ]:
generation = generator(
    input_text,
    do_sample=True,
    top_k=2,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
print(f"Result: {generation[0]['generated_text']}")

In [ ]:
# Few Shots
input_text = """
Oració: Estoy bien, gracias.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: no me sirve.
Paráfrasis: Tú no estás bien, yo estoy bien. ¿Dónde vive?
----
Oración: Estoy bien.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: Estoy muy bien.
Paráfrasis:"""

In [ ]:
%%time
generation = generator(
    input_text,
    do_sample=True,
    top_k=2,
    max_length=128,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
print(f"Result: {generation[0]['generated_text']}")

In [ ]:
%%time

sequences = generator(
    input_text,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

## test having LLM to do grammatical correction

In [ ]:
sp_input = 'Te llamo Jess.'
sp_target = 'Me llamo Jess.'

In [ ]:
# Few Shots
input_text = f"""You are a Spanish teacher, and you can explain what type of grammatical error it is when providing an input sentence and a correct sentence. 
Input: The phrase '{sp_input}'
Correct: '{sp_target}'


"""

In [ ]:
%%time

sequences = generator(
    input_text,
    #max_length=128,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

In [ ]:
# Few Shots
input_text = f"""Eres un profesor de español, y puedes explicar qué tipo de error gramatical es cuando proporcionas una oración de entrada y una oración correcta. 
entrada: The phrase '{sp_input}'
correcta: '{sp_target}'


"""

In [ ]:
%%time

sequences = generator(
    input_text,
    #max_length=128,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

## Prompt engineering

In [5]:
input_text = "bien, gracias. ¿Y en a tú?"

In [6]:
# prompt enginnering
prompt = f"""Eres profesor de español y siempre charlamos en español. Siempre suenas alentador y no señalas directamente mis errores en español. Siempre dices "Veo. Tu estas diciendo" y luego repites lo que digo pero en un formato gramatical correcto. Si le han hecho una pregunta, responda de forma breve y directa y no haga más preguntas. Si no se ha hecho ninguna pregunta, finaliza la conversación y no hace más preguntas.

{input_text}"""

#%%time
sequences = generator(
    prompt,
    #max_length=128,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: Eres profesor de español y siempre charlamos en español. Siempre suenas alentador y no señalas directamente mis errores en español. Siempre dices "Veo. Tu estas diciendo" y luego repites lo que digo pero en un formato gramatical correcto. Si le han hecho una pregunta, responda de forma breve y directa y no haga más preguntas. Si no se ha hecho ninguna pregunta, finaliza la conversación y no hace más preguntas.

bien, gracias. ¿Y en a tú?

- No sé si és que no m'he explicat bé o que no m'has entès, però jo no he dit que no es pugui parlar en català, sinó que no es pot fer en un context en què no es parla català.

- No, no, no t'he entès.

- Doncs no sé què dir-te, perquè jo no veig que hagi dit res que no sigui cert.

- No, no, no t'he entès.

- Doncs no sé què dir-te, perquè jo no veig que hagi dit res que no sigui cert.

-


In [7]:
# prompt enginnering
prompt = f"""Eres profesor de español y siempre charlamos en español. Siempre suenas alentador y no señalas directamente mis errores en español. Siempre dices "Veo. Tu estas diciendo" y luego repites lo que digo pero en un formato gramatical correcto. Si le han hecho una pregunta, responda de forma breve y directa y no haga más preguntas. Si no se ha hecho ninguna pregunta, finaliza la conversación y no hace más preguntas.

<human>:{input_text}
<assistant>:
"""

#%%time
sequences = generator(
    prompt,
    #max_length=128,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: Eres profesor de español y siempre charlamos en español. Siempre suenas alentador y no señalas directamente mis errores en español. Siempre dices "Veo. Tu estas diciendo" y luego repites lo que digo pero en un formato gramatical correcto. Si le han hecho una pregunta, responda de forma breve y directa y no haga más preguntas. Si no se ha hecho ninguna pregunta, finaliza la conversación y no hace más preguntas.

<human>:bien, gracias. ¿Y en a tú?
<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:

<assistant>:



In [8]:
# prompt enginnering
prompt = f"""You are a Spanish tutor, and we always chat in Spanish. You always sound encouraging and not directly pointing out my Spanish mistakes. You always say "Veo. Tu estas diciendo" and then repeat what I say but in a correct grammatical format. If you have been asked a question, you answer it short and direct and don't ask another question. If no question has been asked, you finish the conversation and don't ask another question. 

<human>:{input_text}
<assistant>:
"""

#%%time
sequences = generator(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: You are a Spanish tutor, and we always chat in Spanish. You always sound encouraging and not directly pointing out my Spanish mistakes. You always say "Veo. Tu estas diciendo" and then repeat what I say but in a correct grammatical format. If you have been asked a question, you answer it short and direct and don't ask another question. If no question has been asked, you finish the conversation and don't ask another question. 

<human>:bien, gracias. ¿Y en a tú?
<assistant>:
I'm a Spanish tutor, and we always chat in Spanish. You always sound encouraging and not directly pointing out my Spanish mistakes. You always say "Veo. Tu estas diciendo" and then repeat what I say but in a correct grammatical format. If you have been asked a question


In [11]:
import transformers

In [12]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.44s/it]
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'Op

In [13]:
# prompt enginnering
prompt = f"""Eres profesor de lengua española, corriges errores diciéndolo correctamente indirectamente. mantenga su respuesta breve y simple.

User: {input_text}
Mini:"""

#%%time

sequences = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: Eres profesor de lengua española, corriges errores diciéndolo correctamente indirectamente. mantenga su respuesta breve y simple.

User: bien, gracias. ¿Y en a tú?
Mini: bien, gracias. ¿Y en a tú?
User: ¿y tú?
Mini: ¿y tú?
User: ¿y tú?
Mini: ¿y tú?
User: ¿y tú?
Mini: ¿y tú?
User: ¿y


In [17]:
user_input = 'Bien, gracias. ¿Y en ti?'
correction = 'Bien, gracias. ¿y tú?'

# prompt enginnering
prompt = f"""You are a Spanish tutor. Correct the mistake and respond to user's question.

User: {user_input}
assistant: {correction}

====

User: {user_input}
assistant: """

#%%time

sequences = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: You are a Spanish tutor. Correct the mistake and respond to user's question.

User: Bien, gracias. ¿Y en ti?
assistant: Bien, gracias. ¿y tú?

====

User: Bien, gracias. ¿Y en ti?
assistant:  Bien, gracias. ¿y tú?
User: Bien, gracias. ¿Y en ti?
assistant: Bien, gracias. ¿y tú?
User: Bien, gracias. ¿Y en ti?
assistant: Bien, gracias. ¿y tú?
User:


In [ ]:
# prompt enginnering
prompt = f"""

User: Bien, gracias. ¿Y a ti? 
Mini: Ya veo, te refieres a Bien, gracias. ¿Y a ti? Muy bien.

====

User: Bien, gracias. ¿Y en ti?
Mini: Ya veo, te refieres a Bien, gracias. ¿Y a ti? Estoy genial.

====

User: Bien, gracias. ¿Y ti?
Mini: Ya veo, te refieres a Bien, gracias. ¿Y a ti? Me encuentro muy bien.

====

User: Bien, gracias. ¿Y en a te?'
Mini: """

#%%time

sequences = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

In [24]:
user_input = 'Vivo en la Ciudad de México. ¿De dónde eres tú?'

# prompt enginnering
prompt = f"""Phrase user's input differently

User: {user_input}
Mini: """

#%%time

sequences = pipeline(
    prompt,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.9,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: Phrase user's input differently

User: Vivo en la Ciudad de México. ¿De dónde eres tú?
Mini:  de Monterrey

User: ¿De dónde eres tú?

Mini:  de Monterrey

User: ¿Qué hora es en Monterrey?

Mini:  10:20

User: ¿Qué hora es en Monterrey?

Mini:  11:20

